In [96]:
from collections import deque
import time
import math

In [97]:
class Visitor: # generate unique Visitor
    def __init__(self, id, arrival_time, fast_pass):
        self.id = id # unique
        self.fast_pass = fast_pass # 0 for no fast_pass, 1 for have fast_pass
        self.arrival_time = arrival_time # when they enter the queue
        self.start_time = None # when they start the ride
        self.end_time = None # when they finish the ride
        self.waiting_time = None # waiting time before they get served

    def __repr__(self): # representation method 
        return f"Visitor {self.id, self.arrival_time, self.fast_pass}"


In [98]:
class RideQueues:
    def __init__(self, ride_time, ride_capacity, current_time):
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()
        self.ride_time = ride_time  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take
        self.current_time = current_time  # Simulation time in seconds
        self.total_served = 0  # cumulative number of served visitors 
        self.rides_served = 0 # Cumulative number of rides served in the current simulation, Flag for early termination

    def get_rides_served(self):
        return self.rides_served

    def advance_time(self):
        # Update the current time based on the ride time
        self.current_time += self.ride_time
        print(f"Time advanced to: {self.current_time} seconds.")

    def add_visitor(self, visitor):
        if isinstance(visitor, Visitor):
            if visitor.arrival_time <= self.current_time:
                if visitor.fast_pass == 1:
                    self.fast_pass_queue.append(visitor)
                    print(f"{visitor} added to the fast-pass queue at time {self.current_time}.") # for debugtest
                else:
                    self.regular_queue.append(visitor)
                    print(f"{visitor} added to the regular queue at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at time {self.current_time}.")
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest
        

    def process_queue(self):
        # assumption: ride_capacity is EVEN number; fast-pass vs regular queue each take up 1/2 of ride_capacity
        # operation: take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue)), the rest from regular_queue
        
        
        riding_visitors = [] # list of visitors being served in the current ride
        
        # take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue))
        if self.fast_pass_queue: # while fast_pass_queue has visitors
            for _ in range(min(self.ride_capacity//2, len(self.fast_pass_queue))):
                person = self.fast_pass_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in fast_pass_queue
                    print(f"{person} from Fast Pass Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        fast_pass_filled_ride = len(riding_visitors) # number of fast_pass visitors going to the current ride
        
        # the rest from regular_queue
        if self.regular_queue:
            for _ in range(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue))):
                person = self.regular_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in regular_queue
                    print(f"{person} from Regular Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        if not self.fast_pass_queue and not self.regular_queue:
            print("No one is waiting in the queues at time", self.current_time)
        
        self.rides_served += 1

        # now we update the time
        for visitor in riding_visitors: 
            visitor.start_time = self.current_time  # When visitor starts the ride
            visitor.end_time = self.current_time + self.ride_time
            print(f"{visitor} started at {visitor.start_time} and finished at {visitor.end_time}.") # for debugtest
            self.calculate_waiting_time(visitor)

        # Create outputs: curr_wait_time, crowd_level
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        curr_wait_time = {'fast_pass_queue':math.floor(crowd_level['fast_pass_queue']/self.ride_capacity)*self.ride_time, 
                          'regular_queue':math.floor(crowd_level['regular_queue']/self.ride_capacity)*self.ride_time}

        # and lastly, we update the current time
        self.current_time += self.ride_time
        print(f"The time now is {self.current_time}.")

        return {
            'riding_visitors': riding_visitors, # list of visitors being served at the current ride
            'rides_served': self.rides_served, # flag for early termination
            'total_served': self.total_served, # total number of visitors served up until that moment
            'current_time': self.current_time,
            'crowd_level': crowd_level, # dictionary
            'curr_wait_time': curr_wait_time # dictionary
        }

    def calculate_waiting_time(self, visitor):
        visitor.waiting_time = visitor.start_time - visitor.arrival_time
        print(f"{visitor} waited for {visitor.waiting_time} seconds.")


In [99]:
# Create a RideQueues instance
ride_time = 10  # Each ride takes 10 seconds
ride_capacity = 6  # Max capacity of the ride is 6 people
current_time = 0 # Customisable
ride_queues = RideQueues(ride_time, ride_capacity, current_time)

# Import cust_df ###IMPT, need to link from attendance.ipynb

# For now we create the cust_df manually
import pandas as pd

data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8],
    'arrival_time': [0, 1, 2, 3, 4, 5, 6, 7],
    'fast_pass': [0, 1, 0, 1, 0, 1, 0, 1]
}

cust_df = pd.DataFrame(data)

print(cust_df)


   id  arrival_time  fast_pass
0   1             0          0
1   2             1          1
2   3             2          0
3   4             3          1
4   5             4          0
5   6             5          1
6   7             6          0
7   8             7          1


In [100]:
# Creating a list of Visitor instances from df, need to link from attendance.ipynb

# cust_df columns: id, arrival_time, fast_pass

visitors = []
for index, row in cust_df.iterrows():
    new_visitor = Visitor(id=row['id'], arrival_time=row['arrival_time'], fast_pass=row['fast_pass'])
    visitors.append(new_visitor)

In [101]:
# Trying to define a function that outputs the crowd_level (number of customers in the queue) and waiting_time (expected wait time if one person is to join the queue)

def stat_ride_queues(ride_queues, visitors, threshold): # parameters are: a RideQueues object, a list, threshold is number of rides to process before closing
    # Add customers to the queues
    
    process_queue_stat = {}
    while ride_queues.get_rides_served() <= threshold:
        for visitor in visitors: # add customers to the queue
            ride_queues.add_visitor(visitor)

        result = ride_queues.process_queue() # process the queue
        curr_list_of_visitors = result['riding_visitors']
        process_queue_stat[result['current_time']] = result

        print(f"Riding visitors this round: {result['riding_visitors']}")
        print(f"Total visitorrs served: {result['total_served']}")

        if curr_list_of_visitors: # remove served visitors
            visitors = [visitor for visitor in visitors if visitor not in curr_list_of_visitors] 
    
    return process_queue_stat

In [102]:
# Run the test cases
one_ride_test_result = stat_ride_queues(ride_queues, visitors, 10)
print(one_ride_test_result)


Visitor (1, 0, 0) added to the regular queue at time 0.
Visitor (2, 1, 1) has not yet arrived at time 0.
Visitor (3, 2, 0) has not yet arrived at time 0.
Visitor (4, 3, 1) has not yet arrived at time 0.
Visitor (5, 4, 0) has not yet arrived at time 0.
Visitor (6, 5, 1) has not yet arrived at time 0.
Visitor (7, 6, 0) has not yet arrived at time 0.
Visitor (8, 7, 1) has not yet arrived at time 0.
Visitor (1, 0, 0) from Regular Queue is being served.
total serving so far: 1
No one is waiting in the queues at time 0
Visitor (1, 0, 0) started at 0 and finished at 10.
Visitor (1, 0, 0) waited for 0 seconds.
The time now is 10.
Riding visitors this round: [Visitor (1, 0, 0)]
Total visitorrs served: 1
Visitor (2, 1, 1) added to the fast-pass queue at time 10.
Visitor (3, 2, 0) added to the regular queue at time 10.
Visitor (4, 3, 1) added to the fast-pass queue at time 10.
Visitor (5, 4, 0) added to the regular queue at time 10.
Visitor (6, 5, 1) added to the fast-pass queue at time 10.
Visit